# TD Ameritrade

In [1]:
# Import the libraries
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 
from IPython.display import clear_output
import pandas as pd
import requests
import time
from IPython.display import clear_output

In [2]:
# Client ID
client_id = "YourClientID"

In [3]:
# Endpoint
endpoint = r"https://api.tdameritrade.com/v1/marketdata/quotes"

In [16]:
# Set the ticker names
ticker_names = ["GOOG", "MSFT", "AAPL", "PINS", "TSLA"]
ticker_names = ",".join(ticker_names)

GOOG,MSFT,AAPL,PINS,TSLA


In [ ]:
# Initialise the step count
step_count = 0

# Infinite loop 
while True:
        
    # Make a request
    content = requests.get(url = endpoint, params = {"apikey": client_id, "symbol": ticker_names})

    # Convert data into a json dictionary
    data = pd.DataFrame(content.json())

    # Get the time in epoch
    epoch_time = int(time.time())

    # Loop over the ticker names
    for i_ticker in data:

        # Get the ticker data
        data_temp = data[i_ticker]

        # Connect to the database
        con = psycopg2.connect(database = "TDAMERITRADE", 
                               user = "postgres", 
                               password = "6989490", 
                               host = "127.0.0.1", 
                               port = "5432")

        # Initialize a cursor
        cur = con.cursor()
        
        # Add data to SQL database
        cur.execute("INSERT INTO TD_Ameritrade_2020_July (assetType, assetMainType, cusip, symbol, description, bidPrice, bidSize, bidId, askPrice, askSize, askId, lastPrice, lastSize, lastId, openPrice, highPrice, lowPrice, bidTick, closePrice, netChange, totalVolume, quoteTimeInLong, tradeTimeInLong, mark, exchange, exchangeName, marginable, shortable, volatility, digits, High52Wk, Low52Wk, nAV, peRatio, divAmount, divYield, divDate, securityStatus, regularMarketLastPrice, regularMarketLastSize, regularMarketNetChange, regularMarketTradeTimeInLong, netPercentChangeInDouble, markChangeInDouble, markPercentChangeInDouble, regularMarketPercentChangeInDouble, delayed, time) VALUES {}".format(tuple(data_temp) + (epoch_time,)))

        # Commit and close the connection
        con.commit()
        con.close()

        # Increment the step count
        step_count += 1

        # Report
        clear_output(wait = True)
        print("{} rows of data have been retrieved.".format(step_count))

# The End!